In [1]:
import json
import numpy as np

In [2]:
with open('../../data/raw/test-claims-unlabelled.json', 'rb') as f:
   future_claims = json.load(f)

In [3]:
import pickle
with open('../../predictions/Retrievals/Retrieval_BERT_neg4_reduced_future_FINAL.pickle', 'rb') as f:
  retrievals = pickle.load(f)


In [5]:
with open('../../predictions/Classifications/Classification_BERTFINAL_BERTFINAL5_futurep.pickle', 'rb') as f:
    classif = pickle.load(f)

In [6]:
ENCODING = {'REFUTES': 0, 'DISPUTED': 1, 'NOT_ENOUGH_INFO': 2, 'SUPPORTS': 3}
DECODING = {ENCODING[key]:key for key in ENCODING}

In [7]:
K_RETRIEV = 3
K_CLASSIF = 1

In [8]:
submission = {}
for claim in retrievals:
    
    tmp = {}
    tmp['claim_text'] = future_claims[claim]['claim_text']

    logits = classif[claim]

    sum_logits = [sum([logits[j][i] for j in range(K_CLASSIF)]) for i in range(len(logits[0]))]
    classified_label_index = np.argmax(sum_logits)
    classified_label = DECODING[classified_label_index]

    tmp['claim_label'] = classified_label
    tmp['evidences'] = [x[0] for x in retrievals[claim]['evidences'][:K_RETRIEV]]
    # tmp['evidences'] = [x[0] for x in retrievals[claim][:K_RETRIEV]]

    submission[claim] = tmp

In [9]:
submission

{'claim-2967': {'claim_text': 'The contribution of waste heat to the global climate is 0.028 W/m2.',
  'claim_label': 'REFUTES',
  'evidences': ['evidence-963856', 'evidence-219780', 'evidence-654610']},
 'claim-979': {'claim_text': '“Warm weather worsened the most recent five-year drought, which included the driest four-year period on record in terms of statewide precipitation.',
  'claim_label': 'NOT_ENOUGH_INFO',
  'evidences': ['evidence-178433', 'evidence-126736', 'evidence-1023411']},
 'claim-1609': {'claim_text': 'Greenland has only lost a tiny fraction of its ice mass.',
  'claim_label': 'REFUTES',
  'evidences': ['evidence-1200633', 'evidence-858437', 'evidence-880254']},
 'claim-1020': {'claim_text': '“The global reef crisis does not necessarily mean extinction for coral species.',
  'claim_label': 'NOT_ENOUGH_INFO',
  'evidences': ['evidence-754568', 'evidence-580567', 'evidence-82133']},
 'claim-2599': {'claim_text': 'Small amounts of very active substances can cause large 

In [10]:
with open('../../predictions/test-claims-predictions.json', 'w') as f:
    json.dump(submission, f) 